In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [ ]:
import os
from PIL import Image

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 250, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
OSError: [Errno 9] Bad file descriptor
Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/usr/lib/python3.10/mul

In [ ]:
# Set device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
info=pd.read_csv("/content/a. IDRiD_Disease Grading_Training Labels.csv")
info=info.iloc[: , 0:2]
info.head()

,Image name,Retinopathy grade
0,IDRiD_001,3
1,IDRiD_002,3
2,IDRiD_003,2
3,IDRiD_004,3
4,IDRiD_005,4


In [ ]:
info['category'] = info['Retinopathy grade'].map(lambda x: 1 if x > 0 else 0)
info.head()

,Image name,Retinopathy grade,category
0,IDRiD_001,3,1
1,IDRiD_002,3,1
2,IDRiD_003,2,1
3,IDRiD_004,3,1
4,IDRiD_005,4,1


In [ ]:
testinfo=pd.read_csv("/content/b. IDRiD_Disease Grading_Testing Labels.csv")
testinfo=testinfo.iloc[: , 0:2]
testinfo.head()

,Image name,Retinopathy grade
0,IDRiD_001,4
1,IDRiD_002,4
2,IDRiD_003,4
3,IDRiD_004,4
4,IDRiD_005,4


In [ ]:
testinfo['category'] = testinfo['Retinopathy grade'].map(lambda x: 1 if x > 0 else 0)
testinfo.head()

,Image name,Retinopathy grade,category
0,IDRiD_001,4,1
1,IDRiD_002,4,1
2,IDRiD_003,4,1
3,IDRiD_004,4,1
4,IDRiD_005,4,1


In [ ]:
import shutil
#shutil.copytree('/content/drive/MyDrive/a. Training Set', '/content/train')
#shutil.copytree('/content/drive/MyDrive/b. Testing Set' , '/content/test')

In [ ]:
impathtrain = '/content/train'
impathtest = '/content/test'

In [ ]:
images_dir = impathtrain
output_dir = '/content/traincategory'  # Directory where sorted subfolders will be created

# Iterate over the rows of the DataFrame
for index, row in info.iterrows():
    image_filename = row['Image name'] + ".jpg"
    category = row['category']

    # Create subfolder if it doesn't exist
    category_dir = os.path.join(output_dir, str(category))
    os.makedirs(category_dir, exist_ok=True)

    # Move the image file to the corresponding subfolder
    source_path = os.path.join(images_dir, image_filename)
    destination_path = os.path.join(category_dir, image_filename)
    shutil.move(source_path, destination_path)

FileNotFoundError: ignored

In [ ]:
imagest_dir = impathtest
outputt_dir = '/content/testcategory'  # Directory where sorted subfolders will be created

# Iterate over the rows of the DataFrame
for index, row in testinfo.iterrows():
    image_filename = row['Image name'] + ".jpg"
    category = row['category']

    # Create subfolder if it doesn't exist
    category_dir = os.path.join(outputt_dir, str(category))
    os.makedirs(category_dir, exist_ok=True)

    # Move the image file to the corresponding subfolder
    source_path = os.path.join(imagest_dir, image_filename)
    destination_path = os.path.join(category_dir, image_filename)
    shutil.move(source_path, destination_path)

In [ ]:
# Define the transforms for the images
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet mean and standard deviation
])

In [ ]:
# Load the pre-trained ResNet-50 model
model = models.resnet50(pretrained=True)
num_features = model.fc.in_features  # Get the number of input features for the last layer
model.fc = nn.Linear(num_features, 2)  # Replace the last layer with a binary classifier
model.to(device)
model.eval()

#freezing earlier layers
for param in model.parameters():
    param.requires_grad = False

for param in model.fc.parameters():
    param.requires_grad = True

In [ ]:
# Load your training and test sets using torchvision.datasets.ImageFolder
train_dataset = datasets.ImageFolder("/content/traincategory", transform=transform)
test_dataset = datasets.ImageFolder("/content/testcategory", transform=transform)

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

In [ ]:
# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 20

accumulation_steps = 4  # Accumulate gradients over 4 batches before optimization

for epoch in range(num_epochs):
    running_loss = 0.0
    accumulation_count = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and accumulate gradients
        loss.backward()
        accumulation_count += 1

        if accumulation_count == accumulation_steps:
            # Update weights after accumulating gradients
            optimizer.step()
            optimizer.zero_grad()
            accumulation_count = 0

        running_loss += loss.item()

    # Print training loss for each epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")


Epoch 1/20, Loss: 3.0393
Epoch 2/20, Loss: 2.6476
Epoch 3/20, Loss: 1.3051
Epoch 4/20, Loss: 2.1452
Epoch 5/20, Loss: 2.4654
Epoch 6/20, Loss: 2.0523
Epoch 7/20, Loss: 1.0960
Epoch 8/20, Loss: 0.5734
Epoch 9/20, Loss: 1.1089
Epoch 10/20, Loss: 1.3579
Epoch 11/20, Loss: 0.8176
Epoch 12/20, Loss: 0.4341
Epoch 13/20, Loss: 0.5200
Epoch 14/20, Loss: 0.7950
Epoch 15/20, Loss: 0.8923
Epoch 16/20, Loss: 0.7074
Epoch 17/20, Loss: 0.4935
Epoch 18/20, Loss: 0.3901
Epoch 19/20, Loss: 0.5010
Epoch 20/20, Loss: 0.6539


In [ ]:
model.eval()
correct = 0
total = 0

predicted_labels = []
actual_labels = []

for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)

    # Perform the prediction
    with torch.no_grad():
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

    # Append the predicted and actual labels to the respective lists
    predicted_labels.extend(predicted.cpu().numpy())
    actual_labels.extend(labels.cpu().numpy())

    total += labels.size(0)
    correct += (predicted == labels).sum().item()

# Convert the lists to numpy arrays
predicted_labels = np.array(predicted_labels)
actual_labels = np.array(actual_labels)

# Create the confusion matrix
confusion_mat = confusion_matrix(actual_labels, predicted_labels)

# Print the confusion matrix
print(confusion_mat)


# Print test accuracy
accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

[[28  6]
 [26 43]]
Test Accuracy: 68.93%


# Making a single prediction

In [ ]:
image = Image.open('/content/testcategory/1/IDRiD_001.jpg')
image = transform(image).unsqueeze(0)
with torch.no_grad():
    outputs = model(image)
    _, predicted = torch.max(outputs.data, 1)

# Print the predicted class label
print("Predicted class label:", predicted.item())

Predicted class label: 1
